In [1]:
from functools import reduce
from datetime import datetime

from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np


Prueba de consepto

In [2]:
base_url = 'https://www.infocasas.com.uy'
source_url = 'https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/'
url_page = source_url + 'pagina{}'.format(1)
print(url_page)
#page = urllib.request.urlopen(url_page)
#soup = BeautifulSoup(page, 'html.parser')

#table = soup.find_all('div', attrs={'class': 'propiedades-slider'})

https://www.infocasas.com.uy/venta/casas-apartamento/montevideo/pagina1


In [3]:
neighborhood = [
    [k.text for k in p.find_all('p')] for t in table \
    for p in t.find_all('div')\
    if 'singleLineDots' in p['class']
]
price = [p.text.split()[-1] for t in table \
         for p in t.find_all('div') if 'precio' in p['class']]

desc = [[k.text for k in p.find_all('p')] for t in table \
        for p in t.find_all('div') if 'inDescription' in p['class']]
desc = [k[0] for k in desc]

details = [[d.find_all('span')[0].text for d in p.find_all('div')]\
           for t in table for p in t.find_all('div')\
           if 'contentIcons' in p['class']]
details = pd.DataFrame(details, columns=['rooms', 'bathrooms', 'area_m2'])

data_id = [k.get('data-id', '') for k in table]
data_idproject = [k.get('data-idproyecto', '') for k in table]
link = [base_url + k.find('a')['href'] for k in table]



df = pd.DataFrame(neighborhood, columns=['neighborhood', 'type'])
df['price'] = price
df['desc'] = desc
df['uris'] = link
df['id'] = data_id
df['idproject'] = data_idproject
df = pd.concat([details, df], axis=1)


NameError: name 'table' is not defined

In [4]:
def retrive_data(index,
                 url_base='https://www.infocasas.com.uy',
                 url_filter='/venta/casas-apartamento/montevideo/'):
    url_page = url_base + url_filter + 'pagina{}'.format(index)
    print(url_page)
    try:
        page = urllib.request.urlopen(url_page)
        soup = BeautifulSoup(page, 'html.parser')

        table = soup.find_all('div', attrs={'class': 'propiedades-slider'})
        neighborhood = [
            [k.text for k in p.find_all('p')] for t in table \
            for p in t.find_all('div')\
            if 'singleLineDots' in p['class']
        ]
        price = [p.text.split()[-1] for t in table \
                 for p in t.find_all('div') if 'precio' in p['class']]
        desc = [[k.text for k in p.find_all('p')] for t in table \
                for p in t.find_all('div') if 'inDescription' in p['class']]
        desc = [k[0] for k in desc]
        details = [[d.find_all('span')[0].text for d in p.find_all('div')]\
                   for t in table for p in t.find_all('div')\
                   if 'contentIcons' in p['class']]
        details = pd.DataFrame(details, columns=['rooms', 'bathrooms', 'area_m2'])
        data_id = [k.get('data-id', '') for k in table]
        data_idproject = [k.get('data-idproyecto', '') for k in table]
        link = [url_base + k.find('a')['href'] for k in table]

        df = pd.DataFrame(neighborhood, columns=['neighborhood', 'type'])
        df['price'] = price
        df['desc'] = desc
        df['uris'] = link
        df['id'] = data_id
        df['idproject'] = data_idproject
        df['idproject'] = index
        df = pd.concat([details, df], axis=1)
    except ValueError as e:
        df = pd.DataFrame([])
    
    return df

In [ ]:
nrange = np.arange(1, 941)
indexs = np.random.permutation(nrange)
resume = []
for kidx in indexs:
    resume.append(retrive_data(kidx))

In [7]:
dataset = reduce(lambda x, y: pd.concat([x,y], axis=0), resume)
dataset.reset_index(inplace=True, drop=True)

In [9]:
len(resume)

915

In [8]:
dataset.describe()

,rooms,bathrooms,area_m2,neighborhood,type,price,desc,uris,id,idproject
count,25285,24527,24527,27450,27450,27450,27450,27450,27450,27450
unique,470,4,874,90,13,2239,24631,26058,26058,618
top,2,1,N/A,Pocitos,Apartamentos,120.000,,https://www.infocasas.com.uy/cordn-sur-a-recic...,186014349,
freq,7818,13093,697,4496,18539,355,444,2,2,26797


In [10]:
dataset.to_csv('data/raw_houses_dataset_{}.csv'.format(datetime.today()), index=False)

In [11]:
area = (dataset['area_m2'].str.replace('.', ''))
area[area == 'N/A'] = None
area = area.astype(float)

In [12]:
price = dataset['price'].str.replace('.', '')
price[price == 'Consultar'] = None
price[price == 'Precio'] = None
price = price.astype(float)

In [13]:
rooms = dataset['rooms'].str.replace('+', '')
rooms[rooms == 'Dormitorios'] = None
rooms[rooms == 'Monoambiente'] = 0
rooms = rooms.astype(float)


In [14]:
bathrooms = dataset['bathrooms'].str.replace('+', '')
bathrooms[bathrooms == 'Baños'] = None
bathrooms = bathrooms.astype(float)

In [ ]:
mask = area<5000

In [15]:
dataset.loc[:, 'rooms'] = rooms
dataset.loc[:, 'bathrooms'] = bathrooms
dataset.loc[:, 'area_m2'] = area
dataset.loc[:, 'price'] = price

In [16]:
dataset.to_csv('data/dataset_houses_{}.csv'.format(datetime.today()), index=False)